In [1]:
import pandas as pd

In [2]:
file_type = 'thread'
field_name = 'title'

In [3]:
df = pd.read_csv(f'{file_type}_with_yap.csv')

In [4]:
df_err = df.loc[df[f'{field_name}_dict'].str.contains('error')]

In [5]:
ratio = float(df_err.shape[0]) / df.shape[0]
print(f'ratio of errors {ratio}')

ratio of errors 0.0


In [6]:
df = df.loc[~df[f'{field_name}_dict'].str.contains('error')]

In [7]:
def get_tokenized_from_md_lattice(md_lattice_text,use_lemma=True,keep_only_NN=False):
    tokens = list()
    token_blocks = md_lattice_text.split('\n')
    for token_block in token_blocks:
        token_block_parts = token_block.split('\t')
        if len(token_block_parts) > 2:
            if keep_only_NN and token_block_parts[4] != 'NN':
                continue
            if use_lemma:
                tokens.append(token_block_parts[2])
            else:
                tokens.append(token_block_parts[3])
    return tokens

In [8]:
def replace_words_rule(token_list):
    new_token_list = list()
    replace_words_source = []
    replace_words_target= []
    for token in token_list:
        new_token = token
        try:
            new_token = replace_words_target[replace_words_source.index(token)]
        except:
            pass
        new_token_list.append(new_token)
    reutnr 

In [9]:
def remove_speical_signs(token_list):
    new_token_list = list()
    sign_list = [',','?','!','.']
    for token in token_list:
        new_token = token
        for sign in sign_list:
            new_token = new_token.replace(sign,'')
        new_token_list.append(new_token)
    return new_token_list

In [10]:
def group_tokens_by_list_help(token_list,word_list,gram_size):
    new_token_list = list()
    token_index = 0
    while token_index < len(token_list) - gram_size +1:
        first_token = token_list[token_index]
        indices = [i for i, x in enumerate(word_list[0]) if x == first_token]
        find_one_match = False
        for first_token_index in indices:
            match = True
            for i in range(gram_size):
                if word_list[i][first_token_index] != token_list[token_index+i]:
                    match = False
                    break
            if match:
                find_one_match = True
                new_token = ''
                for i in range(gram_size -1 ):
                    new_token += word_list[i][first_token_index]
                    new_token += ' '
                new_token += word_list[-1][first_token_index]
                new_token_list.append(new_token)
                token_index += gram_size
        if not find_one_match:
            new_token_list.append(first_token)
            token_index += 1
    while token_index < len(token_list):
        new_token_list.append(token_list[token_index])
        token_index += 1
    return new_token_list

In [11]:
def convert_from_phrases_list_word_dict(phrases_list):
    word_lists_dict = dict()
    for phrase in phrases_list:
        phrases_words = phrase.split(' ')
        phrase_len = len(phrases_words)
        if not phrase_len in word_lists_dict:
            word_lists_dict[phrase_len] = list()
            for i in phrase_len:
                word_lists_dict.append(list())
        for i in phrase_len:
            word_lists_dict[phrase_len][i].append(phrases_words[i])
    return word_lists_dict

In [12]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('phrases.txt', 'r', encoding='utf8') as f:
    text = f.read()
phrases_list = set(word for word in text.split('\r\n'))

In [13]:
def group_tokens_by_list(token_list,word_lists_dict):
    gram_sizes = sorted(word_lists.keys, reverse=True)
    for gram_size in gram_sizes:
        token_list = group_tokens_by_list_help(token_list,word_lists_dict[gram_size],gram_size)
    return token_list  

In [14]:
def apply_custom_rules(token_list):
    return token_list

In [15]:
df['token_set'] = df[f'{field_name}_dict'].apply(lambda x: apply_custom_rules(get_tokenized_from_md_lattice(eval(x)['md_lattice'])),False,False)

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,thread_id,title,type,title_dict,token_set
0,0,0,20216916,"עמותות בצל משבר הקורונה - בואו להתעניין, להיעז...",NaN,{'ma_lattice': '0\t1\tעמותות\tעמותה\tNNT\tNNT\...,"[עמותות, ב, צל, משבר, ה, קורונה, -, ב, ה, ואו,..."
1,1,0,20289621,שיבוץ לתפקיד שאני לא רוצה - FXP,NaN,{'ma_lattice': '0\t1\tשיבוץ\tשיבוץ\tNNT\tNNT\t...,"[שיבוץ, ל, ה, תפקיד, ש, אני, לא, רוצה]"
2,2,0,19881195,דרושים מנהלים לפורום צבא וביטחון! - FXP,NaN,{'ma_lattice': '0\t1\tדרושים\tדרש\tBN\tBN\tgen...,"[דרושים, מנהלים, ל, פורום, צבא, ו, ב, ה, יטחון!]"
3,3,0,19801135,השימושון של פורום צבא וביטחון - FXP,NaN,{'ma_lattice': '0\t1\tה\tה\tDEF\tDEF\t_\t1\n0\...,"[ה, שימושון, של, פורום, צבא, ו, ביטחון]"
4,4,0,20246418,יום הזיכרון בFXP - פורום הנצחה והזדהות - FXP,NaN,{'ma_lattice': '0\t1\tיום\tיום\tNNT\tNNT\tgen=...,"[יום, ה, זיכרון, ב, FXP, -, פורום, הנצחה, ו, ה..."


In [17]:
df.iloc[0]['title']

'עמותות בצל משבר הקורונה - בואו להתעניין, להיעזר או לעזור. - FXP'

In [18]:
df.iloc[0]['token_set']

['עמותות',
 'ב',
 'צל',
 'משבר',
 'ה',
 'קורונה',
 '-',
 'ב',
 'ה',
 'ואו',
 'ל',
 'ה',
 'התעניין,',
 'להיעזר',
 'או',
 'ל',
 'ה',
 'עזור.']

In [19]:
['עמותה', 'קורונה', 'התעניין,', 'עזור.']

['עמותה', 'קורונה', 'התעניין,', 'עזור.']

In [20]:
row_number = 0 
ma_lattice_parts = eval(df.iloc[row_number]['title_dict'])['md_lattice'].split('\n')
for part in ma_lattice_parts:
    print(part.split('\t'))

['0', '1', 'עמותות', 'עמותה', 'NN', 'NN', 'gen=F|num=P', '1']
['1', '2', 'ב', 'ב', 'PREPOSITION', 'PREPOSITION', '_', '2']
['2', '3', 'צל', 'צל', 'NNT', 'NNT', 'gen=M|num=S', '2']
['3', '4', 'משבר', 'משבר', 'NNT', 'NNT', 'gen=M|num=S', '3']
['4', '5', 'ה', 'ה', 'DEF', 'DEF', '_', '4']
['5', '6', 'קורונה', 'קורונה', 'NN', 'NN', 'gen=M|num=S', '4']
['6', '7', '-', '_', 'yyDASH', 'yyDASH', '_', '5']
['7', '8', 'ב', 'ב', 'PREPOSITION', 'PREPOSITION', '_', '6']
['8', '9', 'ה', 'ה', 'DEF', 'DEF', '_', '6']
['9', '10', 'ואו', 'ואו', 'INTJ', 'INTJ', '_', '6']
['10', '11', 'ל', 'ל', 'PREPOSITION', 'PREPOSITION', '_', '7']
['11', '12', 'ה', 'ה', 'DEF', 'DEF', '_', '7']
['12', '13', 'התעניין,', 'התעניין,', 'NN', 'NN', 'gen=M|num=P', '7']
['13', '14', 'להיעזר', 'נעזר', 'VB', 'VB', '_', '8']
['14', '15', 'או', 'או', 'CC', 'CC', '_', '9']
['15', '16', 'ל', 'ל', 'PREPOSITION', 'PREPOSITION', '_', '10']
['16', '17', 'ה', 'ה', 'DEF', 'DEF', '_', '10']
['17', '18', 'עזור.', 'עזור.', 'NN', 'NN', 'gen=M|n

In [21]:
corpus = df['token_set'].tolist()

In [22]:
import pickle
with open(f'{file_type}_{field_name}_token_lists.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)